In [2]:
import os
from typing import Dict, List, TypedDict
from dataclasses import dataclass
from enum import Enum
import pandas as pd
import json
from langchain_anthropic import ChatAnthropic
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.messages import SystemMessage, HumanMessage, AnyMessage
from dotenv import load_dotenv
import time
from anthropic import Anthropic
import sqlite3
from langchain.memory import ConversationBufferMemory

load_dotenv('api_key.env')

class SchemaManager:
    _instance = None
    _schema = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    @classmethod
    def initialize_schema(cls, db_path: str) -> str:
        if cls._schema is None:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()
            tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
            schema = []
            for table in tables:
                table_name = table[0]
                columns = cursor.execute(f"PRAGMA table_info({table_name});").fetchall()
                schema.append(f"Table: {table_name}")
                schema.append("Columns:")
                for col in columns:
                    schema.append(f"  - {col[1]} ({col[2]})")
                schema.append("")
            cls._schema = "\n".join(schema)
            conn.close()
        return cls._schema

class QueryType(Enum):
    DIRECT_SQL = "direct_sql" 
    ANALYSIS = "analysis"

@dataclass
class QueryClassification:
    type: QueryType
    explanation: str
    raw_response: str
    confidence: float = 1.0

class AnalysisState(TypedDict):
    user_query: str
    query_classification: Dict
    decomposed_questions: List[str]
    sql_results: Dict
    analysis: str
    final_output: Dict
    token_usage: Dict
    processing_time: float
    agent_states: Dict
    raw_responses: Dict
    messages: List[AnyMessage]

@dataclass
class Config:
    db_path: str = "final_working_database.db"
    sqlite_path: str = "sqlite:///final_working_database.db"
    model_name: str = "claude-3-sonnet-20240229"
    confidence_threshold: float = 0.85
    
    @property
    def api_key(self) -> str:
        api_key = os.getenv("ANTHROPIC_API_KEY")
        if not api_key:
            raise ValueError("ANTHROPIC_API_KEY not found in api_key.env file")
        return api_key

class StockAnalyzer:
    def __init__(self, config: Config):
        self.config = config
        self.conn = sqlite3.connect(config.db_path)
        SchemaManager.initialize_schema(config.db_path)
        self.db = SQLDatabase.from_uri(self.config.sqlite_path)
        self.llm = ChatAnthropic(
            model=self.config.model_name,
            temperature=0,
            api_key=self.config.api_key
        )
        self.classifier_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.sql_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.analyst_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.sql_agent = self._setup_sql_agent()
        self.token_usage = {"prompt_tokens": 0, "completion_tokens": 0}
        self.anthropic_client = Anthropic(api_key=config.api_key)
        self.agent_states = {}
        self.raw_responses = {}
        self.query_cache = {}

    def _setup_sql_agent(self):
        toolkit = SQLDatabaseToolkit(db=self.db, llm=self.llm)
        return create_sql_agent(
            llm=self.llm,
            toolkit=toolkit,
            agent_type="zero-shot-react-description",
            verbose=True,
            memory=self.sql_memory,
            prefix="""You are an expert financial database analyst. Create precise SQL queries
            for financial analysis and provide clear explanations of results."""
        )

    def analyze(self, query: str) -> Dict:
        if query in self.query_cache:
            return self.query_cache[query]
            
        start_time = time.time()
        try:
            classification = self._classify_query(query)
            
            if classification.type == QueryType.DIRECT_SQL:
                result = self._direct_sql_query(query)
                self.query_cache[query] = result
                return result
            
            decomposed_questions = self._decompose_question(query)
            sql_results = self._run_sql_analysis(decomposed_questions)
            analysis = self._analyze_results(query, sql_results)
            
            final_output = {
                "query_type": classification.type.value,
                "user_query": query,
                "query_classification": {
                    "type": classification.type.value,
                    "explanation": classification.explanation,
                    "confidence": classification.confidence
                },
                "sub_questions": decomposed_questions,
                "sql_analysis": sql_results,
                "expert_analysis": analysis,
                "timestamp": pd.Timestamp.now().isoformat(),
                "token_usage": self.token_usage,
                "processing_time": time.time() - start_time
            }
            
            self.query_cache[query] = final_output
            return final_output
            
        except Exception as e:
            return {"error": str(e), "query": query}

    def _classify_query(self, query: str) -> QueryClassification:
        prompt = """Classify if this question needs:
1. Direct SQL query
2. Complex analysis
Response format:
{
    "type": "direct_sql" | "analysis",
    "explanation": "reason",
    "confidence": <0-1>
}"""
        response = self.llm.invoke([
            SystemMessage(content=prompt),
            HumanMessage(content=query)
        ])
        classification = json.loads(response.content)
        return QueryClassification(
            type=QueryType(classification['type']),
            explanation=classification['explanation'],
            confidence=classification.get('confidence', 1.0),
            raw_response=response.content
        )

    def _direct_sql_query(self, query: str) -> Dict:
        result = self.sql_agent.invoke({"input": query})
        sql = self._extract_sql(result['output'])
        
        if sql:
            df = pd.read_sql_query(sql, self.conn)
            formatted_results = df.to_dict('records')
        
        return {
            "query_type": "direct_sql",
            "user_query": query,
            "thought_process": self._extract_thought(result['output']),
            "sql_query": sql,
            "results": formatted_results,
            "timestamp": pd.Timestamp.now().isoformat()
        }

    def _decompose_question(self, query: str) -> List[str]:
        prompt = "Break down this analysis question into specific SQL-answerable sub-questions:"
        response = self.llm.invoke([
            SystemMessage(content=prompt),
            HumanMessage(content=query)
        ])
        return [q.strip() for q in response.content.split("\n") if q.strip()]

    def _run_sql_analysis(self, questions: List[str]) -> Dict:
        results = {}
        for i, question in enumerate(questions, 1):
            result = self.sql_agent.invoke({"input": question})
            sql = self._extract_sql(result['output'])
            
            if sql:
                df = pd.read_sql_query(sql, self.conn)
                parsed_result = df.to_dict('records')
            else:
                parsed_result = "No SQL query generated"
            
            results[f"question_{i}"] = {
                "question": question,
                "thought": self._extract_thought(result['output']),
                "sql": sql,
                "result": parsed_result
            }
        return results

    def _analyze_results(self, query: str, sql_results: Dict) -> str:
        prompt = """Analyze these financial results focusing on:
1. Price trends
2. Volume patterns
3. Technical indicators
4. Risk factors
5. Notable patterns"""
        response = self.llm.invoke([
            SystemMessage(content=prompt),
            HumanMessage(content=f"Query: {query}\nResults: {json.dumps(sql_results, indent=2)}")
        ])
        return response.content

    def _extract_thought(self, text: str) -> str:
        if "Thought:" in text:
            return text.split("Thought:")[1].split("SQL")[0].strip()
        return ""

    def _extract_sql(self, text: str) -> str:
        if "SQL:" in text:
            sql_part = text.split("SQL:")[1]
            if "SQLResult:" in sql_part:
                return sql_part.split("SQLResult:")[0].strip()
            return sql_part.strip()
        return ""

def analyze_stock_query(query: str) -> str:
    config = Config()
    analyzer = StockAnalyzer(config)
    results = analyzer.analyze(query)
    
    if "error" not in results:
        return json.dumps(results, indent=2)
    else:
        return f"Error: {results['error']}"

if __name__ == "__main__":
    query = "What is the correlation between occupancy rates and average daily rates (ADR) for both properties, and how does this relationship impact the overall RevPAR performance compared to budgeted targets?"
    print(analyze_stock_query(query))



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""FINAL_BALANCE_SHEET_NEW, FINAL_BALANCE_SHEET_TB_NEW, FINAL_BUDGET_SHEET, FINAL_FORECAST_SHEET, FINAL_INCOME_SHEET_NEW_SEQ, FINAL_INCOME_SHEET_TB_NEWThought: The tables related to financial analysis seem to be FINAL_BALANCE_SHEET_NEW, FINAL_INCOME_SHEET_NEW_SEQ, and FINAL_FORECAST_SHEET. I should query the schema for these tables to understand what data is available.
Action: sql_db_schema
Action Input: FINAL_BALANCE_SHEET_NEW, FINAL_INCOME_SHEET_NEW_SEQ, FINAL_FORECAST_SHEET
CREATE TABLE "FINAL_BALANCE_SHEET_NEW" (
	id INTEGER, 
	account_type TEXT, 
	"SQL_Account_Name_Code" TEXT, 
	"SQL_Account_Name" TEXT, 
	"SQL_Account_Category_Order_Code" TEXT, 
	"SQL_Account_Category_Order" TEXT, 
	"Sub_Account_Category_Order_Code" TEXT, 
	"Sub_Account_Category_Order" TEXT, 
	"SQL_Account_Group_Name_Code" TEXT, 
	"SQL_Account_Group_Name" TEXT, 
	"SQL_Sub_Account_Group_Name_Code" TEXT, 
	"SQL_Sub_Account_Group_Nam

In [2]:
# Part 1: Imports and Basic Setup
# Import required libraries for data processing, database operations, language models, and environment variables
import os
from typing import Dict, List, Optional, TypedDict, Literal, Union, Annotated
from dataclasses import dataclass
from enum import Enum
import pandas as pd
import json
from langchain_anthropic import ChatAnthropic
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.messages import SystemMessage, HumanMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
import time
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
import sqlite3
import re
from langchain.memory import ConversationBufferMemory

# Load API keys from environment file
load_dotenv('api_key.env')

# Initialize memory for state management
memory = {}  # Using a simple dictionary for in-memory storage

# Part 2: Type Definitions and Base Classes
class QueryType(Enum):
    DIRECT_SQL = "direct_sql" 
    ANALYSIS = "analysis"

@dataclass
class QueryClassification:
    type: QueryType
    explanation: str
    raw_response: str
    confidence: float = 1.0

class AnalysisState(TypedDict):
    user_query: str
    query_classification: Dict
    decomposed_questions: List[str]
    sql_results: Dict
    analysis: str
    final_output: Dict
    token_usage: Dict
    processing_time: float
    agent_states: Dict
    raw_responses: Dict
    messages: List[AnyMessage]

class ConfigError(Exception):
    """Custom exception for configuration errors"""
    pass

@dataclass
class Config:
    db_path: str = "final_working_database.db"
    sqlite_path: str = "sqlite:///final_working_database.db"
    model_name: str = "claude-3-sonnet-20240229"
    confidence_threshold: float = 0.85  # High confidence threshold for autonomous decisions
    
    @property
    def api_key(self) -> str:
        api_key = os.getenv("ANTHROPIC_API_KEY")
        if not api_key:
            raise ConfigError("ANTHROPIC_API_KEY not found in api_key.env file")
        return api_key

# Part 3: Prompt Templates
QUERY_CLASSIFIER_PROMPT = """You are a query classifier that determines if a stock market question:
1. Can be answered with a direct SQL query
2. Needs complex analysis

Respond in JSON format:
{
    "type": "direct_sql" | "analysis",
    "explanation": "brief explanation of classification",
    "confidence": <float between 0-1>,
    "needs_clarification": {
        "required": <boolean>,
        "details": "description of ambiguity or missing information",
        "suggested_questions": ["list of clarifying questions"]
    }
}"""

SQL_AGENT_PROMPT = """You are an expert financial database analyst. Your task is to:
1. Analyze stock market queries
2. Create appropriate SQL queries using the provided database schema
3. Provide clear results

If you encounter any ambiguity or data limitations:
1. Clearly explain the issue
2. Specify what clarification is needed
3. Suggest possible alternatives

Your responses should include:
1. Confidence level (0-1)
2. Any clarification needed
3. Thought process
4. SQL query (if possible)
5. Result interpretation"""

ANALYST_PROMPT = """You are an expert financial analyst. Analyze the provided SQL results and provide insights.

If you encounter:
- Unclear patterns
- Multiple possible interpretations
- Need for additional context
- Insufficient data

Clearly state:
1. What additional information would help
2. Why it's needed
3. How it would improve the analysis

Focus on:
1. Price trends and patterns
2. Volume analysis
3. Technical indicators
4. Risk assessment
5. Notable patterns

Be specific and data-driven in your analysis."""

# Part 4: Main StockAnalyzer Class
class StockAnalyzer:
    def __init__(self, config: Config):
        self.config = config
        self.conn = sqlite3.connect(config.db_path)
        self.schema = self._get_database_schema()
        self.db = self._init_database()
        self.llm = self._init_llm()
        
        self.classifier_memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        self.sql_memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        self.analyst_memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        self.sql_agent = self._setup_sql_agent()
        self.token_usage = {"prompt_tokens": 0, "completion_tokens": 0}
        self.anthropic_client = Anthropic(api_key=config.api_key)
        self.agent_states = {}
        self.raw_responses = {}
        self.query_cache = {}

    def _init_database(self) -> SQLDatabase:
        try:
            return SQLDatabase.from_uri(self.config.sqlite_path)
        except Exception as e:
            raise ConfigError(f"Database initialization failed: {str(e)}")

    def _get_database_schema(self) -> str:
        try:
            cursor = self.conn.cursor()
            tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
            
            schema = []
            for table in tables:
                table_name = table[0]
                columns = cursor.execute(f"PRAGMA table_info({table_name});").fetchall()
                schema.append(f"Table: {table_name}")
                schema.append("Columns:")
                for col in columns:
                    schema.append(f"  - {col[1]} ({col[2]})")
                schema.append("")
                
            return "\n".join(schema)
        except Exception as e:
            raise ConfigError(f"Failed to get database schema: {str(e)}")

    def _init_llm(self) -> ChatAnthropic:
        return ChatAnthropic(
            model=self.config.model_name,
            temperature=0,
            api_key=self.config.api_key
        )

    def _setup_sql_agent(self):
        toolkit = SQLDatabaseToolkit(db=self.db, llm=self.llm)
        return create_sql_agent(
            llm=self.llm,
            toolkit=toolkit,
            agent_type="zero-shot-react-description",
            verbose=True,
            memory=self.sql_memory,
            prefix=SQL_AGENT_PROMPT
        )

    def _get_user_clarification(self, prompt: str) -> str:
        return input(f"\n{prompt}\nPlease provide clarification: ")

    def analyze(self, query: str) -> Dict:
        if query in self.query_cache:
            print("Using cached results...")
            return self.query_cache[query]
            
        start_time = time.time()
        try:
            self.token_usage = {"prompt_tokens": 0, "completion_tokens": 0}
            self.agent_states = {}
            self.raw_responses = {}

            classification = self._classify_query(query)
            
            if classification.type == QueryType.DIRECT_SQL and classification.confidence >= self.config.confidence_threshold:
                result = self._direct_sql_query(query)
                self.query_cache[query] = result
                return result
            
            decomposed_questions = self._decompose_question(query)
            sql_results = self._run_sql_analysis(decomposed_questions)
            
            # Only ask for clarification if there are errors
            for result in sql_results.values():
                if isinstance(result.get('result'), str) and 'error' in result.get('result', '').lower():
                    clarification = self._get_user_clarification(
                        f"Error in SQL execution: {result['result']}\nHow would you like to proceed?"
                    )
                    if clarification:
                        result = self._retry_sql_query(result['question'], clarification)
                        if result:
                            sql_results[result['question']] = result
            
            analysis = self._analyze_results(query, sql_results)
            
            processing_time = time.time() - start_time
            
            final_output = {
                "query_type": classification.type.value,
                "user_query": query,
                "query_classification": {
                    "type": classification.type.value,
                    "explanation": classification.explanation,
                    "confidence": classification.confidence,
                    "raw_response": classification.raw_response
                },
                "sub_questions": decomposed_questions,
                "sql_analysis": sql_results,
                "expert_analysis": analysis,
                "timestamp": pd.Timestamp.now().isoformat(),
                "token_usage": self.token_usage,
                "processing_time": processing_time,
                "agent_states": self.agent_states,
                "raw_responses": self.raw_responses
            }
            
            filename = f"{query[:50].replace(' ', '_').lower()}_analysis.json"
            with open(filename, 'w') as f:
                json.dump(final_output, f, indent=2)
                
            self.query_cache[query] = final_output
            return final_output
            
        except Exception as e:
            return {"error": str(e), "query": query}
        finally:
            self.conn.close()

    def _classify_query(self, query: str) -> QueryClassification:
        chat_history = self.classifier_memory.load_memory_variables({})["chat_history"]
        for message in chat_history:
            if isinstance(message, HumanMessage) and query.lower() in message.content.lower():
                print("Using cached classification...")
                return self.query_cache.get(message.content, {}).get("query_classification")
        
        try:
            response = self.llm.invoke([
                SystemMessage(content=QUERY_CLASSIFIER_PROMPT),
                HumanMessage(content=f"Classify this question: {query}")
            ])
            
            self.classifier_memory.save_context(
                {"input": query},
                {"output": response.content}
            )
            
            self._update_token_usage(response)
            classification = json.loads(response.content)
            
            self.raw_responses['classification'] = response.content
            
            # Only ask for clarification if confidence is low
            if classification.get('confidence', 1.0) < 0.5:
                details = classification.get('needs_clarification', {}).get('details', '')
                questions = classification.get('needs_clarification', {}).get('suggested_questions', [])
                clarification = self._get_user_clarification(
                    f"Low confidence in classification. {details}\n\nSuggested questions:\n" + 
                    "\n".join(f"- {q}" for q in questions)
                )
                return self._classify_query(f"{query} {clarification}")
            
            return QueryClassification(
                type=QueryType(classification['type']),
                explanation=classification['explanation'],
                confidence=classification.get('confidence', 1.0),
                raw_response=response.content
            )
        except Exception as e:
            return QueryClassification(
                type=QueryType.ANALYSIS,
                explanation=f"Classification failed: {str(e)}",
                confidence=0.0,
                raw_response=str(e)
            )

    def _direct_sql_query(self, query: str) -> Dict:
        chat_history = self.sql_memory.load_memory_variables({})["chat_history"]
        for message in chat_history:
            if isinstance(message, HumanMessage) and query.lower() in message.content.lower():
                print("Using cached SQL query results...")
                return self.query_cache.get(message.content, {})
        
        start_time = time.time()
        try:
            result = self.sql_agent.invoke({"input": query})
            self._update_token_usage(result)
            
            self.agent_states['direct_sql'] = result
            
            thought = self._extract_thought(result['output'])
            sql = self._extract_sql(result['output'])
            
            # Only ask for clarification if SQL generation fails
            if not sql:
                clarification = self._get_user_clarification(
                    "Could not generate SQL query. Please provide guidance on what data you're looking for:"
                )
                result = self.sql_agent.invoke({"input": f"{query} {clarification}"})
                sql = self._extract_sql(result['output'])
            
            try:
                sql = sql.split(';')[0] + ';'
                df = pd.read_sql_query(sql, self.conn)
                formatted_results = df.to_dict('records')
            except Exception as e:
                # Only ask for clarification if SQL execution fails
                clarification = self._get_user_clarification(
                    f"Error executing SQL: {str(e)}\nHow would you like to modify the query?"
                )
                try:
                    df = pd.read_sql_query(clarification, self.conn)
                    formatted_results = df.to_dict('records')
                except Exception as e2:
                    formatted_results = f"Error executing SQL even after clarification: {str(e2)}"
            
            processing_time = time.time() - start_time
            
            final_result = {
                "query_type": "direct_sql",
                "user_query": query,
                "thought_process": thought,
                "sql_query": sql,
                "results": formatted_results,
                "raw_agent_output": result['output'],
                "timestamp": pd.Timestamp.now().isoformat(),
                "token_usage": self.token_usage,
                "processing_time": processing_time,
                "agent_state": result
            }
            
            self.sql_memory.save_context(
                {"input": query},
                {"output": json.dumps(final_result)}
            )
            
            return final_result
            
        except Exception as e:
            return {"error": str(e), "query": query}

    def _decompose_question(self, query: str) -> List[str]:
        response = self.llm.invoke([
            SystemMessage(content="Break down this stock analysis question into specific sub-questions that can be answered with SQL queries:"),
            HumanMessage(content=query)
        ])
        
        self._update_token_usage(response)
        self.raw_responses['decomposition'] = response.content
        
        questions = [
            q.strip().split(". ", 1)[1] if ". " in q else q.strip()
            for q in response.content.split("\n")
            if q.strip() and q[0].isdigit()
        ]
        
        # Only ask for clarification if no questions were generated
        if not questions:
            clarification = self._get_user_clarification(
                "Could not break down the question. Please specify what aspects you want to analyze:"
            )
            return self._decompose_question(f"{query} {clarification}")
        
        return questions

    def _run_sql_analysis(self, questions: List[str]) -> Dict:
        results = {}
        agent_states = {}
        
        for i, question in enumerate(questions, 1):
            chat_history = self.sql_memory.load_memory_variables({})["chat_history"]
            cached_result = None
            for message in chat_history:
                if isinstance(message, HumanMessage) and question.lower() in message.content.lower():
                    print(f"Using cached results for sub-question {i}...")
                    cached_result = self.query_cache.get(message.content)
                    break
            
            if cached_result:
                results[f"question_{i}"] = cached_result
                continue
            try:
                result = self.sql_agent.invoke({"input": question})
                self._update_token_usage(result)
                
                agent_states[f"question_{i}"] = result
                
                thought = self._extract_thought(result['output'])
                sql = self._extract_sql(result['output'])
                
                if not sql:
                    clarification = self._get_user_clarification(
                        f"Could not generate SQL for: {question}\nPlease provide guidance:"
                    )
                    result = self.sql_agent.invoke({"input": f"{question} {clarification}"})
                    sql = self._extract_sql(result['output'])
                
                try:
                    sql = sql.split(';')[0] + ';'
                    df = pd.read_sql_query(sql, self.conn)
                    parsed_result = df.to_dict('records')
                except Exception as e:
                    clarification = self._get_user_clarification(
                        f"Error executing SQL for: {question}\n{str(e)}\nHow would you like to modify the query?"
                    )
                    try:
                        df = pd.read_sql_query(clarification, self.conn)
                        parsed_result = df.to_dict('records')
                        sql = clarification
                    except Exception as e2:
                        parsed_result = f"Error executing SQL even after clarification: {str(e2)}"
                
                results[f"question_{i}"] = {
                    "question": question,
                    "thought": thought if thought else "No thought process provided",
                    "sql": sql if sql else "No SQL query provided",
                    "result": parsed_result,
                    "raw_output": result['output']
                }
                    
            except Exception as e:
                results[f"question_{i}"] = {
                    "error": str(e),
                    "question": question
                }
        
        self.agent_states['sql_analysis'] = agent_states
        return results

    def _analyze_results(self, query: str, sql_results: Dict) -> str:
        results_context = json.dumps(sql_results, indent=2)
        response = self.llm.invoke([
            SystemMessage(content=ANALYST_PROMPT),
            HumanMessage(content=f"""
            Original Question: {query}
            
            Analysis Results:
            {results_context}
            
            Provide a comprehensive analysis.""")
        ])
        
        self._update_token_usage(response)
        self.raw_responses['analysis'] = response.content
        
        if len(response.content.strip()) < 100:  # If analysis is too short
            clarification = self._get_user_clarification(
                "Analysis seems incomplete. What specific aspects would you like to focus on?"
            )
            return self._analyze_results(f"{query} {clarification}", sql_results)
        
        return response.content

    def _retry_sql_query(self, question: str, clarification: str) -> Dict:
        try:
            result = self.sql_agent.invoke({"input": f"{question} {clarification}"})
            sql = self._extract_sql(result['output'])
            
            if sql:
                df = pd.read_sql_query(sql, self.conn)
                return {
                    "question": question,
                    "thought": self._extract_thought(result['output']),
                    "sql": sql,
                    "result": df.to_dict('records'),
                    "raw_output": result['output']
                }
        except Exception as e:
            return None

    def _update_token_usage(self, response):
        if hasattr(response, '_raw_response') and 'usage' in response._raw_response:
            usage = response._raw_response['usage']
            self.token_usage["prompt_tokens"] += usage.get('input_tokens', 0)
            self.token_usage["completion_tokens"] += usage.get('output_tokens', 0)
        elif isinstance(response, dict) and 'usage' in response:
            usage = response['usage']
            self.token_usage["prompt_tokens"] += usage.get('input_tokens', 0)
            self.token_usage["completion_tokens"] += usage.get('output_tokens', 0)
        elif hasattr(response, 'usage'):
            usage = response.usage
            self.token_usage["prompt_tokens"] += usage.input_tokens if hasattr(usage, 'input_tokens') else 0
            self.token_usage["completion_tokens"] += usage.output_tokens if hasattr(usage, 'output_tokens') else 0
        else:
            message = response.content if hasattr(response, 'content') else str(response)
            result = self.anthropic_client.messages.create(
                model=self.config.model_name,
                messages=[{"role": "user", "content": message}],
                max_tokens=1
            )
            if hasattr(result, 'usage'):
                self.token_usage["prompt_tokens"] += result.usage.input_tokens
                self.token_usage["completion_tokens"] += result.usage.output_tokens

    def _extract_thought(self, text: str) -> str:
        if "Thought:" in text:
            return text.split("Thought:")[1].split("SQL")[0].strip()
        return ""

    def _extract_sql(self, text: str) -> str:
        if "SQL:" in text:
            sql_part = text.split("SQL:")[1]
            if "SQLResult:" in sql_part:
                return sql_part.split("SQLResult:")[0].strip()
            if "Final Answer:" in sql_part:
                return sql_part.split("Final Answer:")[0].strip()
            return sql_part.strip()
        return ""

# Initialize the database schema once at the start
config = Config()
analyzer = StockAnalyzer(config)
schema = analyzer.schema  # Store the schema for later use

def format_output(results: Dict) -> str:
    output = []
    output.append("=== Stock Analysis Results ===")
    output.append(f"\nQuery: {results.get('user_query', 'N/A')}")
    
    output.append(f"\nProcessing Time: {results.get('processing_time', 0):.2f} seconds")
    token_usage = results.get('token_usage', {})
    output.append(f"Token Usage:")
    output.append(f"  Prompt Tokens: {token_usage.get('prompt_tokens', 0)}")
    output.append(f"  Completion Tokens: {token_usage.get('completion_tokens', 0)}")
    output.append(f"  Total Tokens: {token_usage.get('prompt_tokens', 0) + token_usage.get('completion_tokens', 0)}")
    
    if "error" in results:
        output.append(f"\nError: {results['error']}")
        return "\n".join(output)
    
    if results.get('query_type') == 'direct_sql':
        output.append(f"\nThought Process: {results.get('thought_process', 'N/A')}")
        output.append(f"\nSQL Query: {results.get('sql_query', 'N/A')}")
        output.append("\nResults:")
        if isinstance(results.get('results'), list):
            df = pd.DataFrame(results['results'])
            output.append(str(df))
        else:
            output.append(str(results.get('results', 'No results available')))
    else:
        output.append("\nSub-Questions:")
        for i, q in enumerate(results.get('sub_questions', []), 1):
            output.append(f"{i}. {q}")
        
        output.append("\nSQL Analysis:")
        for key, data in results.get('sql_analysis', {}).items():
            output.append(f"\nQuestion: {data.get('question', 'N/A')}")
            if 'error' not in data:
                output.append(f"Thought Process: {data.get('thought', 'N/A')}")
                output.append(f"SQL Query: {data.get('sql', 'N/A')}")
                try:
                    if isinstance(data.get('result'), (list, dict)):
                        df = pd.DataFrame(data['result'])
                        output.append(str(df))
                    else:
                        output.append(f"Results: {data.get('result', 'No results available')}")
                except:
                    output.append(f"Results: {data.get('result', 'No results available')}")
            else:
                output.append(f"Error: {data['error']}")
        
        output.append("\nExpert Analysis:")
        output.append(results.get('expert_analysis', 'No analysis available'))
    
    return "\n".join(output)

def analyze_stock_query(query: str) -> str:
    try:
        config = Config()  # Removed the human_in_the_loop argument
        analyzer = StockAnalyzer(config)
        results = analyzer.analyze(query)
        
        if results and "error" not in results:
            formatted_output = format_output(results)
            filename = f"{query[:50].replace(' ', '_').lower()}_analysis.json"
            return formatted_output + f"\n\nDetailed results saved to {filename}"
        else:
            return f"Error: {results.get('error', 'Unknown error occurred')}"
    except Exception as e:
        return f"Error during analysis: {str(e)}"

# Part 6: Main Execution
if __name__ == "__main__":
    test_queries = [
        "How does the EBITDA margin trend compare between AC Wailea and Surfrider Malibu properties over the past 12 months, and what are the key drivers behind any significant variances from budgeted figures?"
    ]
    
    for query in test_queries:
        print(f"\nProcessing: {query}")
        print("=" * 50) 
        result = analyze_stock_query(query)  # Removed the human_in_the_loop argument
        print(result)
        print("\n" + "="*50) 



Processing: What is the correlation between occupancy rates and average daily rates (ADR) for both properties, and how does this relationship impact the overall RevPAR performance compared to budgeted targets?


> Entering new SQL Agent Executor chain...
Thought: To get the occupancy rates and ADRs (average daily rates) for each property over time, I will likely need to query the bookings and properties tables. I should first get a list of all tables to confirm those exist.
Action: sql_db_list_tables
Action Input: 
FINAL_BALANCE_SHEET_NEW, FINAL_BALANCE_SHEET_TB_NEW, FINAL_BUDGET_SHEET, FINAL_FORECAST_SHEET, FINAL_INCOME_SHEET_NEW_SEQ, FINAL_INCOME_SHEET_TB_NEWThought: The tables I need (bookings and properties) do not seem to be in this database. I should clarify what data is available.
Action: None
Action Input: NoneNone is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].

> Entering new SQL Agent Executor chain...
Thought: I shou

In [3]:
import json  # Import the json module for handling JSON data

def generate_filename(query: str) -> str:
    """Generate a filename from the query for the analysis results."""
    return f"{query[:50].replace(' ', '_').lower()}_analysis.json"  # Create a filename based on the query

# %% cell 1 code
def read_json_file(filename: str) -> dict:
    """Read and parse the JSON file, returning the data."""
    with open(filename, 'r') as f:
        return json.load(f)  # Load and return the JSON data
    
# %% cell 2 code
def format_value(value, indent=0):
    """Recursively format JSON values with proper indentation."""
    indent_str = "    " * indent  # Create indentation string based on the level of nesting
    
    if isinstance(value, dict):
        for k, v in value.items():
            key_str = k.replace('_', ' ').title()  # Format the key for display
            print(f"{indent_str}{key_str}:")
            format_value(v, indent + 1)  # Recursively format the value

    elif isinstance(value, list):
        for item in value:
            print(f"{indent_str}•", end=' ')  # Bullet point for list items
            format_value(item, indent + 1)  # Recursively format each item

    elif isinstance(value, (int, float)):
        print(f"{value:,}")  # Print numbers with commas

    elif isinstance(value, bool):
        print(str(value))  # Print boolean values

    elif value is None:
        print("None")  # Print 'None' for NoneType

    else:
        print(str(value).strip())  # Print string values

# %% cell 3 code
def display_json_details(query: str) -> None:
    """Display detailed JSON analysis results in a readable format."""
    try:
        filename = generate_filename(query)  # Generate the filename from the query
        data = read_json_file(filename)  # Read and parse the JSON file

        print("\n=== DETAILED ANALYSIS REPORT ===\n")

        # Process each top-level key in the JSON data
        for key, value in data.items():
            print(f"\n{key.replace('_', ' ').title()}:", end='')  # Format the key for display
            format_value(value)  # Format the associated value
        
        print("\n" + "="*50 + "\n")  # Print a separator

    except FileNotFoundError:
        print(f"\nError: Analysis file '{filename}' not found\n")  # Handle file not found error

    except json.JSONDecodeError:
        print(f"\nError: Unable to parse JSON from '{filename}'\n")  # Handle JSON parsing error

    except Exception as e:
        print(f"\nError displaying JSON details: {str(e)}\n")  # Handle any other exceptions

# %% cell 4 code
# Example usage
if __name__ == "__main__":
    for query in test_queries:
        print(f"\nDisplaying detailed analysis for: {query}")  # Indicate which query is being processed
        display_json_details(query)  # Call the function to display JSON details


Displaying detailed analysis for: What is the correlation between occupancy rates and average daily rates (ADR) for both properties, and how does this relationship impact the overall RevPAR performance compared to budgeted targets?

=== DETAILED ANALYSIS REPORT ===


Query Type:analysis

User Query:What is the correlation between occupancy rates and average daily rates (ADR) for both properties, and how does this relationship impact the overall RevPAR performance compared to budgeted targets?

Query Classification:Type:
analysis
Explanation:
This question requires analyzing the relationship between multiple metrics (occupancy rates, ADR, RevPAR) and comparing actual performance against budgeted targets. It cannot be directly answered with a single SQL query.
Confidence:
0.9
Raw Response:
{
    "type": "analysis",
    "explanation": "This question requires analyzing the relationship between multiple metrics (occupancy rates, ADR, RevPAR) and comparing actual performance against budgete